In [ ]:
import scanpy as sc
import scvi
import anndata
import os
import pandas as pd
import matplotlib.pyplot as plt

# Set random seed for reproducibility
scvi.settings.seed = 42

# Define file paths
file_paths = [
    "/scratch/project_2008506/WMB-10Xv2-Isocortex-1-raw.h5ad",
    "/scratch/project_2008506/WMB-10Xv2-Isocortex-2-raw.h5ad",
    "/scratch/project_2008506/WMB-10Xv2-Isocortex-3-raw.h5ad",
    "/scratch/project_2008506/WMB-10Xv2-Isocortex-4-raw.h5ad",
]

# Load and prepare datasets
adatas = []
for path in file_paths:
    print(f"Loading {path}")
    adata = sc.read_h5ad(path)
    # Extract sample name as batch info
    sample_name = os.path.basename(path).replace('-raw.h5ad', '')
    adata.obs['batch'] = sample_name
    adatas.append(adata)

# Concatenate all datasets
combined = anndata.concat(
    adatas,
    join='outer',  # keep all genes
    label='batch',
    index_unique='-'  # add unique suffix to cell names
)

print(f"Combined shape: {combined.shape}")

In [ ]:
#Remove dead cells
combined.var["mt"] = combined.var_names.str.startswith("mt-")  # Identify mitochondrial genes
sc.pp.calculate_qc_metrics(combined, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

# Filter out cells with >5% mitochondrial content
mito_threshold = 5  # Percentage threshold
filtered_cells = combined.obs["pct_counts_mt"] < mito_threshold

# Apply filtering
combined = combined[filtered_cells, :].copy()

In [ ]:
# Basic quality control filtering
sc.pp.filter_cells(combined, min_counts=1000)  # Filter cells with few counts
print("Filtering cells done")
sc.pp.filter_genes(combined, min_cells=10)     # Filter genes detected in few cells
print("Filtering genes done")
# Normalize and log transform for initial analysis
sc.pp.normalize_total(combined, target_sum=1e4)  # Normalize to 10,000 counts per cell
sc.pp.log1p(combined)                           # Log transform the data

print(f"After preprocessing: {combined.shape}")

#Save
combined.write_h5ad('/scratch/project_2008506/4sample_concatenated_normalized_isocortex.h5ad')


In [ ]:
#PCA neighbours umap
sc.pp.pca(combined, n_comps=50)  # Run initial PCA with 50 components
print("PCA calculated..")
sc.pp.neighbors(combined)
print("Neighbours calculated..")
sc.tl.umap(combined)  # Compute UMAP
print("UMAP calculated..")

#Save
combined.write_h5ad('/scratch/project_2008506/4sample_concatenated_PCA_isocortex.h5ad')

# Generate UMAP svg plot of batch. This plot will saved in svg so that individual points are grouped together. Good format for saving large UMAPs
ax = sc.pl.umap(combined, color=["batch"], title="UMAP by Leiden cluster", show=False)

# Rasterize scatter points manually
for artist in ax.collections:
    artist.set_rasterized(True)

# Save as SVG with rasterized scatter points
plt.savefig("/scratch/project_2008506/4sample_before_integration_batch.svg", dpi=600)
plt.show()

In [ ]:
#Calculate Leiden
import scanpy as sc
combined = sc.read_h5ad('/scratch/project_2008506/4sample_concatenated_PCA_isocortex.h5ad')
sc.tl.leiden(combined, resolution=0.6)  # Adjust resolution if needed

In [ ]:
#Plot leiden clusters similar to mouse data (!!size=1 argument)
# Generate UMAP svg plot of batch
ax = sc.pl.umap(combined, size = 1, color=["leiden"], title="UMAP by Leiden cluster", show=False)

# Rasterize scatter points manually
for artist in ax.collections:
    artist.set_rasterized(True)

# Save as SVG with rasterized scatter points
plt.savefig("/scratch/project_2008506/4sample_before_integration_leiden.svg", dpi=600, bbox_inches="tight")
plt.show()

In [ ]:
#Get leiden cluster abundances (as a barplot)
import pandas as pd
import matplotlib.pyplot as plt

# Count number of cells per Leiden cluster
cluster_counts = combined.obs["leiden"].value_counts().sort_index()

# Create a histogram/bar plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(cluster_counts.index, cluster_counts.values, color="royalblue", edgecolor="black")

# Formatting
ax.set_xlabel("Leiden Cluster", fontsize=12, fontweight="bold")
ax.set_ylabel("Number of Cells", fontsize=12, fontweight="bold")
ax.set_title("Cluster Abundance in Combined Dataset", fontsize=14, fontweight="bold")
ax.tick_params(axis="x", rotation=45)  # Rotate x-axis labels for better visibility

# Save the plot
save_path = "/scratch/project_2008506/leiden_cluster_abundance"
plt.savefig(f"{save_path}.svg", dpi=600, bbox_inches="tight")  # Save as SVG
plt.savefig(f"{save_path}.png", dpi=300, bbox_inches="tight")  # Save as PNG
plt.savefig(f"{save_path}.pdf", dpi=600, bbox_inches="tight")  # Save as PDF

# Show the plot
plt.show()

In [ ]:
#Create and plot cross tables (cross-tabulation) of leiden clusters in different samples
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Compute cross-tabulation
ctab = pd.crosstab(combined.obs["batch"], combined.obs["leiden"])

# Create binary and log2-transformed versions
ctab_binary = (ctab > 0).astype(int)
ctab_log = np.log2(ctab + 1)

# Set up the figure with two subplots (side by side)
fig, axes = plt.subplots(1, 2, figsize=(16, 8))  # 1 row, 2 columns

### 🔹 Plot 1: Binary Heatmap ###
sns.heatmap(ctab_binary.T, annot=False, fmt="d", cmap="Greens", ax=axes[0], 
            square=True, linewidths=0.5, linecolor="gray", 
            cbar_kws={"shrink": 0.8, "aspect": 40})

# Customize labels
axes[0].set_title("Binary Presence Heatmap", fontsize=14, fontweight="bold")
axes[0].set_xlabel("Batch", fontsize=12, fontweight="bold")
axes[0].set_ylabel("Leiden Cluster", fontsize=12, fontweight="bold")

### 🔹 Plot 2: Log2 Heatmap ###
sns.heatmap(ctab_log.T, annot=False, fmt=".2f", cmap="magma", ax=axes[1], 
            square=True, linewidths=0.5, linecolor="gray", 
            cbar_kws={"shrink": 0.8, "aspect": 40})

# Customize labels
axes[1].set_title("Log2 Transformed Heatmap", fontsize=14, fontweight="bold")
axes[1].set_xlabel("Batch", fontsize=12, fontweight="bold")
axes[1].set_ylabel("Leiden Cluster", fontsize=12, fontweight="bold")

# Adjust layout for better spacing
plt.tight_layout()

# Save the figure
save_path = "/scratch/project_2008506/CTAB_binary_vs_log"
fig.savefig(f"{save_path}.svg", dpi=600, bbox_inches="tight")  # Vectorized SVG
fig.savefig(f"{save_path}.png", dpi=300, bbox_inches="tight")  # High-quality PNG
fig.savefig(f"{save_path}.pdf", dpi=600, bbox_inches="tight")  # Publication-ready PDF

# Show the plots
plt.show()


In [ ]:
#Plot cell types based on a few positive markers
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt

# Define marker genes for each cell type
marker_dict = {
    "Inhibitory_Score": ["Gad1", "Gad2", "Pvalb", "Sst", "Vip", "Nkx2-1"],  # GABAergic neurons
    "Excitatory_Score": ["Slc17a7", "Slc17a6", "Cux2", "Rorb", "Bcl11b"],  # Glutamatergic neurons
    "Astrocyte_Score": ["Aldh1l1", "Slc1a3", "Aqp4", "S100b", "Gja1"],  # Astrocytes
    "Oligodendrocyte_Score": ["Mog", "Mbp", "Cnp", "Plp1", "Olig1"],  # Oligodendrocytes
    "Microglia_Score": ["Cx3cr1", "P2ry12", "Tmem119", "Aif1"],  # Microglia
    "Endothelial_Score": ["Pecam1", "Cldn5", "Kdr"],  # Blood-brain barrier cells
    "Pericyte_Score": ["Pdgfrb", "Des", "Anpep"],  # Pericytes
}

# Compute cell type scores
for cell_type, markers in marker_dict.items():
    existing_markers = [gene for gene in markers if gene in combined.var_names]
    
    if len(existing_markers) == 0:
        print(f"⚠️ Warning: No valid markers found for {cell_type}. Skipping.")
        continue

    # Compute mean expression per cell
    combined.obs[cell_type] = np.mean(combined[:, existing_markers].X, axis=1)

# Define cell type labels dynamically
cell_types = list(marker_dict.keys())
titles = [f"UMAP: {ct.replace('_', ' ')}" for ct in cell_types]  # Improve title formatting

# Generate UMAP plots for each cell type
ax = sc.pl.umap(combined, color=cell_types, size=10, cmap="viridis",
                title=titles, show=False)

# Rasterize scatter points manually for large datasets
for axis in ax:
    for artist in axis.collections:
        artist.set_rasterized(True)

# Save the plots with proper formatting
save_path = "/scratch/project_2008506/cell_type_scores"
plt.savefig(f"{save_path}.svg", dpi=600, bbox_inches="tight")  # Save as SVG
plt.savefig(f"{save_path}.png", dpi=300, bbox_inches="tight")  # Save as PNG
plt.savefig(f"{save_path}.pdf", dpi=600, bbox_inches="tight")  # Save as PDF

# Show the figure
plt.show()

In [ ]:
#Use Chao2 estimator on the presence/absence data (not sure if this works, I did these in R)
# Compute Chao2 estimator
S_obs = ctab_binary.sum(axis=1).sum()  # Total observed species (sum across samples)
Q1 = (ctab_binary.sum(axis=0) == 1).sum()  # Number of species appearing in exactly 1 sample
Q2 = (ctab_binary.sum(axis=0) == 2).sum()  # Number of species appearing in exactly 2 samples

# Avoid division by zero in case Q2 is 0
if Q2 > 0:
    S_Chao2 = S_obs + (Q1**2) / (2 * Q2)
else:
    S_Chao2 = S_obs  # If Q2=0, Chao2 falls back to observed richness